In [1]:
import math

import matplotlib.pyplot as plt
import numpy as np
import os
import time

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
import qiskit as qiskit
import qiskit.quantum_info as qi
from qiskit import QuantumCircuit, transpile, Aer, IBMQ

import ramsey_experiment
import estimator

# Loading your IBM Quantum account(s)
IBMQ.save_account(os.environ.get('IBM_KEY'), overwrite=True)
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-research')
import Ramsey_ExperimentV2
import random
import importlib
from tqdm import tqdm
ramsey_experiment = importlib.reload(Ramsey_ExperimentV2)
RamseyExperiment = ramsey_experiment.RamseyExperiment
RamseyBatch = ramsey_experiment.RamseyBatch

ibmqfactory.load_account:WARNING:2024-04-02 11:49:54,295: Credentials are already in use. The existing account in the session will be replaced.


In [2]:
import numpy as np
import math
n = 4
total_shots = 200
T = np.pi
measurements = 50
delay = np.linspace(0, T, measurements)

W = [random.gauss(0, 3) for _ in range(n)]
L = [random.gauss(3, 2) for _ in range(n)]
J = [random.gauss(0, 3) for _ in range(n - 1)]

# L = [3,3,3]
# J = [2,2]
# L = [0,0,0]
# W = [4,4,4]


# delay = [0.3]
batch_x, batch_y = ramsey_experiment.ramsey_global(n,total_shots,delay,L,W,J)
rot = 0.23
batch_x_rot, batch_y_rot = ramsey_experiment.ramsey_global_with_rot(n,total_shots,delay,L,W,J, rot)
batch_x_det, batch_y_det, batch_x_cross, batch_y_cross = ramsey_experiment.ramsey_local(n,total_shots,delay,L,W,J)


In [3]:
ramsey_experiment = importlib.reload(Ramsey_ExperimentV2)
RamseyExperiment = ramsey_experiment.RamseyExperiment
RamseyBatch = ramsey_experiment.RamseyBatch
estimator = importlib.reload(estimator)

## qubit by qubit


In [33]:

guessed_decay,guessed_W,guessed_J = estimator.one_by_one_fit(batch_x_det, batch_y_det, batch_x_cross, batch_y_cross)
print("guessed decay: ", guessed_decay)
print("real decay: ", L)
print("guessed frequency: ", guessed_W)
print("real frequency: ", W)
print("guessed J: ", guessed_J)
print("real J: ", J)

guessed = np.concatenate([guessed_decay,guessed_W,guessed_J])
real_parameters = np.concatenate([L,W,J])
mse = np.mean((guessed - real_parameters)**2)
print("error: ", np.sqrt(mse))
print("error in percent: ", np.sqrt(mse)/np.mean(np.abs(real_parameters)) * 100)

guessed decay:  [4.729545206525623, 4.987670990804653]
real decay:  [4.225127482704364, 5.284195258818687]
guessed frequency:  [-0.7555012938838058, -6.751969062748306]
real frequency:  [-0.29853973665521444, -6.595509516937884]
guessed J:  [0.989158045550349]
real J:  [0.8419625073606084]
error:  0.3456367637706465
error in percent:  10.021167282113504


## global

In [32]:
neighbors = 1
guessed_decay,guessed_W,guessed_J = estimator.full_complex_fit(batch_x, batch_y, neighbors=neighbors)

print("guessed decay: ", guessed_decay)
print("real decay: ", L)
print("guessed frequency: ", guessed_W)
print("real frequency: ", W)
print("guessed J: ", guessed_J)
print("real J: ", J)

guessed = np.concatenate([guessed_decay,guessed_W,guessed_J])
real_parameters = np.concatenate([L,W,J])
mse = np.mean((guessed - real_parameters)**2)
print("error: ", np.sqrt(mse))
print("error in percent: ", np.sqrt(mse)/np.mean(np.abs(real_parameters)) * 100)
print("number of neighbors: ", neighbors)

guessed decay:  [3.55746154 4.60718631]
real decay:  [4.225127482704364, 5.284195258818687]
guessed frequency:  [ 1.46387118 -4.77003041]
real frequency:  [-0.29853973665521444, -6.595509516937884]
guessed J:  [-2.49547672]
real J:  [0.8419625073606084]
error:  1.922554849175537
error in percent:  55.74130350731734
number of neighbors:  1


## global Noise

In [59]:

neighbors = 0
guessed_decay,guessed_W, guessed_J = estimator.full_complex_fit_test(batch_x_rot, batch_y_rot, batch_x, batch_y, rot1=rot)

print("guessed decay: ", guessed_decay)
print("real decay: ", L)
print("guessed frequency: ", guessed_W)
print("real frequency: ", W)
print("guessed J: ", guessed_J)
print("real J: ", J)

guessed = np.concatenate([guessed_decay,guessed_W,guessed_J])
real_parameters = np.concatenate([L,W,J])
mse = np.mean((guessed - real_parameters)**2)
print("error: ", np.sqrt(mse))
print("error in percent: ", np.sqrt(mse)/np.mean(np.abs(real_parameters)) * 100)
print("number of neighbors: ", neighbors)

guessed decay:  [0.39618939 0.04854328]
real decay:  [0.0, 0.0]
guessed frequency:  [-0.32757984 -0.41600841]
real frequency:  [0.0, 0.0]
guessed J:  [-1.79817608]
real J:  [-2.5506421130238763]
error:  0.4485307478982868
error in percent:  87.92506514497592
number of neighbors:  0


In [40]:
import Symbolic.symbolic_evolution as symbolic_evolution
importlib.reload(symbolic_evolution)
symbolic_exp = symbolic_evolution.get_expectation_values_exp(n, neighbors=neighbors)


In [42]:
import matplotlib.pyplot as plt

data = []
for i in range(len(delay)):
    data.append(batch_x.RamseyExperiments[i].get_n_nearest_neighbors(neighbors))
    data.append(batch_y.RamseyExperiments[i].get_n_nearest_neighbors(neighbors))
data = np.concatenate(data)

model = []
guessed = []
for t in delay:
    for i in range(2*(n+n-1)):
        model.append(symbolic_evolution.set_parameters(symbolic_exp[i],W,J,L,t))
        guessed.append(symbolic_evolution.set_parameters(symbolic_exp[i], guessed_W, guessed_J, guessed_decay,t))


# plt.plot(data, model_func(delay,guessed_decay,guessed_W,guessed_J), 'o')
plt.plot(data, model, 'o', label="model")
plt.plot(data, guessed, 'o', label="guessed")
plt.plot(np.linspace(-0.5,0.5,40), np.linspace(-0.5,0.5,40), 'r')
plt.grid()
plt.legend()
# print(z_points - model_func(delay,L,W,J))

KeyboardInterrupt: 